In [1]:
import joblib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import torch
import pandas as pd
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering

# Any results you write to the current directory are saved as output.

/kaggle/input/covidd/comm_use_subset.csv
/kaggle/input/covidd/biorxiv_data.csv
/kaggle/input/articles/AllArticles.csv
/kaggle/input/bertuncaselargesquad/config.json
/kaggle/input/bertuncaselargesquad/vocab.txt
/kaggle/input/bertuncaselargesquad/pytorch_model.bin


In [2]:
print(os.listdir('/kaggle/input/bertuncaselargesquad'))

['config.json', 'vocab.txt', 'pytorch_model.bin']


In [3]:
import torch
import pandas as pd
import numpy as np
from transformers import BartForConditionalGeneration, BartConfig, BartTokenizer

class Summary_Model(object):
    def __init__(self, summary_model_path, token_path, device):
        self.summary_model_path=summary_model_path
        self.token_model_path=token_path
        self.model=None
        self.tokenizer=None
        self.device=device
        self.create_models()

    def create_models(self):
        self.model = BartForConditionalGeneration.from_pretrained(self.summary_model_path)
        self.tokenizer = BartTokenizer.from_pretrained(self.token_model_path)
        self.model = self.model.to(self.device)

    def get_summary(self, data, count):
        ### Top confident answer to article
        total_abstract = ''
        for i in range(len(data[:count])):
            abss, ans = data.loc[i, ['answer_sent', 'Answer']].values
            total_abstract += (abss + ".")
        ARTICLE_TO_SUMMARIZE = total_abstract

        ### Token the article, if larger than 1024, then split the article
        tokens = self.tokenizer.tokenize(ARTICLE_TO_SUMMARIZE)
        max_seq_length = 1024
        longer = 0
        all_tokens = []
        if len(tokens) > 1024:
            for i in range(0, len(tokens), max_seq_length):
                tokens_a = tokens[i:i + max_seq_length]
                one_token = self.tokenizer.batch_encode_plus([tokens_a], max_length=1024, return_tensors='pt')
                all_tokens.append(one_token)

        Summary_text = []

        ## decode the model output as summary text
        def decode_text(sum_ids):
            text = ''
            for g in sum_ids:
                text = text + self.tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False)
            return text

        ## Summary model
        model = self.model.to(self.device)
        model.eval()
        Summary_text = ''
        for inputs in all_tokens:
            summary_ids = model.generate(inputs['input_ids'].to(self.device), num_beams=2, max_length=1000,
                                         early_stopping=True)
            Summary_text = Summary_text + " " + decode_text(summary_ids)

        if Summary_text == '':
            Summary_text = "Can't find summary of answer"
        
        return Summary_text



In [4]:
import torch
import pandas as pd
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering
from tqdm import notebook

class QuestionAnswering(object):
    def __init__(self, bert_model_path, token_model_path, device):
        self.bert_model_path=bert_model_path
        self.token_model_path=token_model_path
        self.model=None
        self.tokenizer=None
        self.device=device
        self.create_models()
        self.sum_model=Summary_Model(summary_model_path=Bart, token_path=Bart, device=device)


    def create_models(self):
        self.model= BertForQuestionAnswering.from_pretrained(self.bert_model_path)
        self.tokenizer=BertTokenizer.from_pretrained(self.token_model_path)
        self.model=self.model.to(self.device)
        
    def get_answer_sentence(self, text, answer):
        text=text.lower()
        answer=answer.lower()
        split_byans = text.split(answer)
        if len(split_byans)==1:
            return split_byans[0][split_byans[0].rfind(". ")+1:]+" "+answer
        else: 
            return split_byans[0][split_byans[0].rfind(". ")+1:]+" "+answer+split_byans[1][:split_byans[1].find(". ")+1]

    def create_answer_text(self, tokens, start=0, stop=-1):
        tokens = tokens[start: stop]
        if '[SEP]' in tokens:
            sepind = tokens.index('[SEP]')
            tokens = tokens[sepind+1:]
        txt = ' '.join(tokens)
        txt = txt.replace(' ##', '')
        txt = txt.replace('##', '')
        txt = txt.strip()
        txt = " ".join(txt.split())
        txt = txt.replace(' .', '.')
        txt = txt.replace('( ', '(')
        txt = txt.replace(' )', ')')
        txt = txt.replace(' - ', '-')
        txt_list = txt.split(' , ')
        txt = ''
        nTxtL = len(txt_list)
        if nTxtL == 1:
            return txt_list[0]
        newList =[]
        for i,t in enumerate(txt_list):
            if i < nTxtL -1:
                if t[-1].isdigit() and txt_list[i+1][0].isdigit():
                    newList += [t,',']
                else:
                    newList += [t, ', ']
            else:
                newList += [t]
        return ''.join(newList)

    def train_model_func(self, question, document):
        token_ids = self.tokenizer.encode(question, document)
        tokens = self.tokenizer.convert_ids_to_tokens(token_ids)
        answers = []
        nWords = len(document.split())
        cs = []
        if len(token_ids) > 512:
            #divIndex = len(token_ids) // 2
            divIndex =int(np.ceil(nWords/2))
            all_sp = document.split()
            docs = [' '.join(all_sp[:divIndex]), ' '.join(all_sp[divIndex:])]
            token_ids = [self.tokenizer.encode(question, doc) for doc in docs]
        else:
            token_ids = [token_ids]

        for token_idp in token_ids:
            tokens = self.tokenizer.convert_ids_to_tokens(token_idp)
            sep_index = token_idp.index(self.tokenizer.sep_token_id)
            

            # Because of CLS
            questP = sep_index + 1
            secp = len(token_idp) - questP

            seq_types = [0] * questP + [1] * secp

            if len(seq_types)<512:

                start_index, end_index = self.model(torch.tensor([token_idp]).to(self.device),
                                                token_type_ids=torch.tensor([seq_types]) \
                                                .to(self.device))
            else:
                start_index, end_index = self.model(torch.tensor([token_idp[:512]]).to(self.device),
                                                    token_type_ids=torch.tensor([seq_types[:512]]) \
                                                    .to(self.device))
            start_index = start_index[:,1:-1]
            end_index = end_index[:,1:-1]

            answer_start = torch.argmax(start_index)
            answer_end = torch.argmax(end_index)

            answer = self.create_answer_text(tokens, answer_start, answer_end+2)

            answers.append(answer)

            c = start_index[0, answer_start].item() + end_index[0, answer_end].item()
            cs.append(c)

        maxC = max(cs)
        maxC_ind = cs.index(maxC)
        answer = answers[maxC_ind]
        return answer, maxC

    def train(self, documents, questions):
        df_dicts = {}
        summaries={}
        for question in questions:
            ids = []
            answers = []
            confs = []
            abstracts = []
            sent_ans=[]
            for document, abstract, id in notebook.tqdm(documents):
                answer, c = self.train_model_func(question, document)
                if answer!='':
                    ans_sent=self.get_answer_sentence(document, answer)
                    sent_ans.append(ans_sent)
                    ids.append(id)
                    answers.append(answer)
                    confs.append(c)
                    abstracts.append(abstract)
            
            
            question_df = pd.DataFrame({'Id': ids, 'abstract': abstracts, 'Answer': answers, 'answer_sent':sent_ans, 'conf': confs})
            question_df = question_df.sort_values(by=['conf'], ascending=False).reset_index(drop=True)
            summary=self.sum_model.get_summary(question_df, 100)
            df_dicts[question] = question_df
            summaries[question]=summary
            # df_lists.append(question_df)
        return df_dicts, summaries

In [5]:
Bert_config='/kaggle/input/bertuncaselargesquad'
use_cuda=torch.cuda.is_available()
device=torch.device('cuda' if use_cuda else 'cpu')
token_p='bert-large-uncased-whole-word-masking-finetuned-squad'
Bart='bart-large-cnn'
#token_path='f{Bert_config}/vocab.txt'

In [6]:
question_model=QuestionAnswering(bert_model_path=Bert_config, token_model_path=token_p, device=device)

In [7]:
# questions=['What is known about transmission,incubation,and environmental stability',
#     'What do we know about COVID-19 risk factors',
#     'What do we know about virus genetics, origin, and evolution',
#     'What do we know about vaccines and therapeutics',
#     'What has been published about medical care',
#     'What do we know about non-pharmaceutical interventions',
#     'What do we know about diagnostics and surveillance',
#     'What has been published about ethical and social science considerations',
#     'What has been published about information sharing and inter-sectoral collaboration']
# # questions=[questions[0]]
# # questions

In [8]:
questin_m=['How does temperature and humidity affect the tramsmission',
          'What do we know about natural history, transmission, and diagnostics',
          'What is the range of incubation period',
          'How long can 2019-nCoV remain viable on inanimate, environmental, or common surfaces',
          'How does smoking affect',
          'How does pregnancy affect patients',
          'How does heart disease affect patients',
          'Can animals transmit',
          'What animal did 2019-nCoV come from']

In [9]:
questin_m2=['What drugs or therapies are being investigated',
          'What antiviral drug clinical trials have been completed',
          'How is artificial intelligence being used in real time health delivery',
          'What adjunctive or supportive methods can help patients',
          'What are most important barriers to compliance',
           'What diagnostic tests (tools) exist or are being developed to detect 2019-nCoV',
          'What is being done to increase testing capacity or throughput',
          'What collaborations are happening within the research community',
           'What factors contribute to rumors and misinformation',
           'How do pandemics affect the physical and/or psychological health of doctors and nurses'
           ]

In [10]:
questin_m3=["What have we learned about infection prevention and control",
          'What is tools and studies to monitor phenotypic change and potential adaptation of the virus',
          'How does hypertension affect patients',
          'What effors are being done in asia to prevent further outbreaks',
          'What should we do to prevent',
           'When can we use the vaccines',
          'Does virus die in hot weather',
          'Can I be affected by the virus by touching the surfaces',
           'Does mask useful',
           'do disinfectants kill the virus',
            'When will it end',
            'Which country was most affected',
            'Which country was less affected',
           ]

In [11]:
# questin_m4=['What drugs or therapies are being investigated',
#           'What antiviral drug clinical trials have been completed',
#           'How is artificial intelligence being used in real time health delivery',
#           'What adjunctive or supportive methods can help patients',
#           'What are most important barriers to compliance',
#            'What diagnostic tests (tools) exist or are being developed to detect 2019-nCoV',
#           'What is being done to increase testing capacity or throughput',
#           'What collaborations are happening within the research community',
#            'What factors contribute to rumors and misinformation',
#            'How do pandemics affect the physical and/or psychological health of doctors and nurses'
#            ]

In [12]:
questin_m

['How does temperature and humidity affect the tramsmission',
 'What do we know about natural history, transmission, and diagnostics',
 'What is the range of incubation period',
 'How long can 2019-nCoV remain viable on inanimate, environmental, or common surfaces',
 'How does smoking affect',
 'How does pregnancy affect patients',
 'How does heart disease affect patients',
 'Can animals transmit',
 'What animal did 2019-nCoV come from']

In [13]:
ddir='/kaggle/input/articles/AllArticles.csv'

df=joblib.load(ddir)
df=df[df['is_covid19']==True]
#df=df[:1000]
df.shape
#texts=df.abstract.values
texts=df.body_text.values
ids=df.Id.values
absts=df.abstract.values

del df

documents=[(texts[i],absts[i], ids[i]) for i in range(len(texts))]
del texts
del ids
del absts

In [14]:
import gc
gc.collect()

0

In [15]:
# df.head()

In [16]:
# dicts,sums=question_model.train(documents,questions_m)

In [17]:
# sums['What is known about transmission,incubation,and environmental stability']

In [18]:
# len(dicts)

In [19]:
# dicts['What is known about transmission,incubation,and environmental stability'].head(10)

In [20]:
# joblib.dump(dicts, '/kaggle/working/Newdictionaries')

In [21]:
# joblib.dump(sums, '/kaggle/working/Newsummaries')

In [22]:
# ndicts,nsums=question_model.train(documents,questin_m)

In [23]:
# joblib.dump(ndicts, '/kaggle/working/NewAddidionaldictionaries')

In [24]:
# joblib.dump(nsums, '/kaggle/working/NewAddidionalsummaries')

In [25]:
# ndicts2,nsums2=question_model.train(documents,questin_m2)

In [26]:
# joblib.dump(ndicts2, '/kaggle/working/NewAddidionaldictionaries2')

In [27]:
# joblib.dump(nsums2, '/kaggle/working/NewAddidionalsummaries2')

In [28]:
ndicts3,nsums3=question_model.train(documents,questin_m3)

In [29]:
joblib.dump(ndicts3, '/kaggle/working/NewAddidionaldictionaries3')

['/kaggle/working/NewAddidionaldictionaries3']

In [30]:
joblib.dump(nsums3, '/kaggle/working/NewAddidionalsummaries3')

['/kaggle/working/NewAddidionalsummaries3']

In [31]:
# pd.set_option('display.max_colwidth',100)

In [32]:
# dicts['What is known about transmissio,incubation,and environmental stability?'].head(10)

In [33]:
# doc=[t for t,a,i in documents if i=='cb3974f33a1d54369d630e62052d603148bb12e7']

In [34]:
# doc[0]